# Curso de aprendizaje automatizado
PCIC, UNAM

Machine Learning

Rodrigo S. Cortez Madrigal

<img src="https://www.iimas.unam.mx/wp-content/uploads/2023/11/Logo-pagina-ok.png" alt="Logo IIMAS" width="200" />  


### Tarea 1: Clasificador bayesiano ingenuo

Un equipo de biólogos está estudiando dos especies de aves que habitan en una región montañosa.
Estas especies tienen preferencias distintas por ciertas condiciones ambientales, lo que influye en su
distribución geográfica. Para caracterizar los lugares donde han sido observadas, los investigadores
registran las siguientes variables:

- x1: Temperatura media del hábitat (◦C)
- x2: Altitud relativa del hábitat (m)
- y: Especie

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import plotly.express as px

In [2]:
# Descargar el conjunto de datos
url = "https://turing.iimas.unam.mx/~gibranfp/cursos/aprendizaje_automatizado/data/aves.csv"
data = pd.read_csv(url, sep=',', index_col=0, dtype={'y': 'int', 'X1': 'float64', 'X2': 'float64'})
data = data.dropna()

In [3]:
species = data['y'].unique()
print(f"Numero de especies: {len(species)}")

Numero de especies: 2


In [4]:
data.head()

,x1,x2,y
0,7.606876,11.421913,0
1,8.300079,12.352260,0
2,6.833771,10.770320,0
3,7.349341,10.903545,0
4,7.442586,11.730364,0


In [5]:
target = data['y'].value_counts()
fig = px.pie(values=target, title='Distribución de clases', names={0: 'Species 1', 1: 'Species 2'})
fig.show()

In [6]:
# Separar características y etiquetas
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

# Dividir el conjunto de datos en entrenamiento (60%) y temporal (40%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=0)

# Dividir el conjunto temporal en validación (50% de 40% = 20%) y prueba (50% de 40% = 20%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)

In [7]:
from sklearn.naive_bayes import MultinomialNB

# Entrenar clasificadores bayesianos ingenuos
gnb = MultinomialNB()

gnb.fit(X_train, y_train)

# Predecir y reportar el porcentaje de predicciones correctas en entrenamiento y validación
y_train_pred_gnb = gnb.predict(X_train)
y_val_pred_gnb = gnb.predict(X_val)

train_accuracy_gnb = accuracy_score(y_train, y_train_pred_gnb) * 100
val_accuracy_gnb = accuracy_score(y_val, y_val_pred_gnb) * 100
test_accuracy_gnb = accuracy_score(y_test, gnb.predict(X_test)) * 100

print(f"MultinomialNB - Precisión en entrenamiento: {train_accuracy_gnb:.2f}%")
print(f"MultinomialNB - Precisión en validación: {val_accuracy_gnb:.2f}%")
print(f"MultinomialNB - Precisión en prueba: {test_accuracy_gnb:.2f}%")

MultinomialNB - Precisión en entrenamiento: 98.42%
MultinomialNB - Precisión en validación: 97.50%
MultinomialNB - Precisión en prueba: 98.75%


In [8]:
from CustomNB import GaussianNaiveBayes
import numpy as np

model = GaussianNaiveBayes()
model.fit(np.array(X_train), np.array(y_train), method='MLE')

In [9]:
y_train_pred_cnb = model.predict(np.array(X_train))
y_val_pred_cnb = model.predict(np.array(X_val))

train_accuracy_cnb = accuracy_score(y_train, y_train_pred_cnb) * 100
val_accuracy_cnb = accuracy_score(y_val, y_val_pred_cnb) * 100
test_accuracy_cnb = accuracy_score(y_test, model.predict(np.array(X_test))) * 100

print(f"CustomNB - Precisión en entrenamiento: {train_accuracy_cnb:.2f}%")
print(f"CustomNB - Precisión en validación: {val_accuracy_cnb:.2f}%")
print(f"CustomNB - Precisión en prueba: {test_accuracy_cnb:.2f}%")

CustomNB - Precisión en entrenamiento: 83.75%
CustomNB - Precisión en validación: 82.75%
CustomNB - Precisión en prueba: 83.75%


Me parece extraño que dados los datos y su naturaleza, el modelo Gaussiano no sea el más adecuado para este problema. 

Esto podria ser debido a que la distribución de los datos no es Gaussiana, o que la varianza de los datos no es constante.

In [10]:
# Busquemos una explicación

import plotly.graph_objects as go

fig1 = px.histogram(data, x='x1', nbins=30, title='Distribución de la Temperatura Media del Hábitat (x1)', marginal='box')
fig1.show()

fig2 = px.histogram(data, x='x2', nbins=30, title='Distribución de la Altitud Relativa del Hábitat (x2)', marginal='box')
fig2.show()

# Estadísticas descriptivas
print(data[['x1', 'x2']].describe())


                x1           x2
count  2000.000000  2000.000000
mean      7.992537    11.037689
std       0.684231     1.369196
min       5.337936     6.596961
25%       7.531162    10.032349
50%       8.000111    11.032112
75%       8.468020    12.051890
max      10.423600    15.558738


In [16]:
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import numpy as np

# Supongamos que 'data' es tu DataFrame con los datos
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

# Aplicar una transformación logarítmica a las características
transformer = FunctionTransformer(np.log1p, validate=True)
X = transformer.transform(X)

# Escalar los datos
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Dividir los datos en conjuntos de entrenamiento, validación y prueba
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=0)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)

# Crear y entrenar el modelo
model = GaussianNB()
model.fit(X_train, y_train)

# Realizar predicciones
y_train_pred = model.predict(X_train)
y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)

# Calcular la precisión
train_accuracy = accuracy_score(y_train, y_train_pred) * 100
val_accuracy = accuracy_score(y_val, y_val_pred) * 100
test_accuracy = accuracy_score(y_test, y_test_pred) * 100

# Imprimir los resultados
print(f"GaussianNB - Precisión en entrenamiento: {train_accuracy:.2f}%")
print(f"GaussianNB - Precisión en validación: {val_accuracy:.2f}%")
print(f"GaussianNB - Precisión en prueba: {test_accuracy:.2f}%")


scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')

# Imprimir los resultados de la validación cruzada
print(f"Precisión media en validación cruzada: {scores.mean() * 100:.2f}%")
print(f"Desviación estándar de la precisión: {scores.std() * 100:.2f}%")

GaussianNB - Precisión en entrenamiento: 83.67%
GaussianNB - Precisión en validación: 82.75%
GaussianNB - Precisión en prueba: 84.75%
Precisión media en validación cruzada: 83.25%
Desviación estándar de la precisión: 1.78%


/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/sklearn/utils/validation.py:2732: UserWarning:

X has feature names, but FunctionTransformer was fitted without feature names



Pues esta raro 